In [ ]:
import re

In [ ]:
def translate(expression):
    negation = ''
    if expression.startswith('_not_'):
        negation = 'not'
        expression = expression.replace('_not_', '').strip()

    expression_parts = expression.split(' ', 2)
    if len(expression_parts) not in [1, 3]:
        raise Exception('Error: Syntax')
    if len(expression_parts) == 1:
        return expression
    
    field, operator, operand = expression_parts[0].strip(), expression_parts[1].strip(), expression_parts[2].strip()
    if operator == '_eq_':
        command = f'{field} == {operand}'
    elif operator == '_dif_':
        command = f'{field} != {operand}'
    elif operator == '_gt_':
        command = f'{field} > {operand}'
    elif operator == '_lt_':
        command = f'{field} < {operand}'
    elif operator == '_ct_':
        command = f'{field}.str.lower().str.contains({operand})'
    elif operator == '_in_':
        command = f'{field}.isin({operand})'

    else:
        raise Exception('Error: operator')
    
    return f'{negation} {command}'.strip()

In [11]:
def parentheses(expression):
    blocks = {}
    count = 1
    while True:
        groups = re.findall('\([^\(\)]*\)', expression)
        if not groups:
            block_id = '_b0'
            blocks[block_id] = expression
            expression = block_id
            break
            
        for gr in groups:
            block_id = f'b{count}'
            blocks[block_id] = gr #[1:-1]
            expression = expression.replace(gr, block_id)
            count += 1
    
    return blocks

In [12]:
def logical(expression):
    expression = expression.replace('(', '').replace(')', '')
    blocks = {}
    for bl in re.split('or | and', expression):
        blocks[bl.strip()] = bl.strip()
    
    return blocks
    

In [61]:
pt = '^_#_b\d{1,2}_#_$'

re.match(pt, '_#_b13_#_'.strip())



<re.Match object; span=(0, 9), match='_#_b13_#_'>

In [13]:
def transl(expr):
    return expr.replace('&&', 'and').replace('||', 'or')

In [25]:
expression = "(a > 1 || b gt 5) && (c !eq 8 && (d < 0 || e ct rt))"
par = parentheses(expression)
print(par)
for p in par:
    par[p] = transl(par[p])

par_list = list(par.keys())

i = 0
expr = '_b0'
while par_list:
    bl = par_list[i]
    if bl in expr:
        expr = expr.replace(bl, par[bl])
        par_list.pop(i)
        i = 0
    else:
        i += 1
        if i >= len(par_list):
            i = 0
print(par)


{'b1': '(a > 1 || b gt 5)', 'b2': '(d < 0 || e ct rt)', 'b3': '(c !eq 8 && b2)', '_b0': 'b1 && b3'}
{'b1': '(a > 1 or b gt 5)', 'b2': '(d < 0 or e ct rt)', 'b3': '(c !eq 8 and b2)', '_b0': 'b1 and b3'}


In [ ]:
expression = "a _ct_ 'b' and _not_ c _in_ [1,3,4] and ((f _lt_ 5 and m _ct_ 'hg') or g _dif_ 8)"
print(expression)
#expression = "_not_ a _dif_ x"
par = parentheses(expression)
for p in par:
    blocks = logical(par[p])

    for bl in blocks:
        blocks[bl] = translate(blocks[bl])

    for bl in blocks:
        par[p] = par[p].replace(bl, blocks[bl])

p_list = list(par.keys())
index = 0
while len(p_list) > 0:
    p = p_list[index]
    if p in expression:
        expression = expression.replace(p, f'{par[p]}')
        p_list.pop(index)
        index = 0
    else:
        index += 1

print(expression)

In [ ]:
txt = 'a ct 6 and (b > 7 or (c < 4 and (d != 9 or e ew "ca"))'

In [ ]:
parenthesis(txt)

In [ ]:
logical('(d != 9 or e ew "ca")')

In [ ]:
def parser(expression):
    blocks = {}
    count = 0
    while True:
        groups = re.findall('\([^\(\)]*\)', expression)
        if not groups:
            field, operator, operand = expression.split(' ', 2)
            block_id = f'b{count}'
            blocks[block_id] = translate(field, operator, operand)
            expression = block_id
            break
            
        for gr in groups:
            field, operator, operand = gr[1:-1].split(' ', 2)
            block_id = f'b{count}'
            blocks[block_id] = '(' + translate(field, operator, operand) + ')'
            expression = expression.replace(gr, block_id)
            count += 1

    block_id_list = [key for key in blocks.keys()]
    i = 0
    while True:
        if len(block_id_list) == 0:
            break
        b = block_id_list[i]
        if b in expression:
            expression = expression.replace(b, blocks.get(b))
            block_id_list.pop(i)
            i = 0
        else:
            print(len(block_id_list), i)
            i = (i+1) if len(block_id_list) > i+1 else 0
    
    return expression

In [ ]:
txt = 'a ct 6 and (b > 7 or (c < 4) - (d != 9))'

In [ ]:
parser(txt)

In [ ]:
help(list().pop)

In [ ]:
expr = '(5 _vezes_ 3) _mais_ 1 _menos_ (4 _vezes_ (7 _menos_ 1))'

In [ ]:
expr2 = expr.replace('(', '[').replace(')', ']')

In [ ]:
expr2

In [ ]:
p = re.findall('\[[^\[\]]*\]', expr2)

In [ ]:
p

In [ ]:
expr3 = expr2.replace(p[0], '(5 * 3)').replace(p[1], '(7 - 1)')

In [ ]:
expr3

In [ ]:
p = re.findall('\[[^\[\]]*\]', expr3)

In [ ]:
p

In [ ]:
expr4 = expr3.replace(p[0], '(4 * (7 - 1))')

In [ ]:
expr4

In [ ]:
expr5 = expr4.replace(expr4, '(5 * 3) + 1 - (4 * (7 - 1))')

In [ ]:
expr5